<a href="https://colab.research.google.com/github/bluetinue/Bert_Medical/blob/main/%E5%9F%BA%E4%BA%8Ebert%E7%9A%84%E9%97%A8%E8%AF%8A%E7%A7%91%E5%AE%A4%E6%8C%82%E5%8F%B7%E5%88%86%E7%B1%BB%E9%A2%84%E6%B5%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 导包
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.optim import AdamW
from torch.utils.data import DataLoader,Dataset
from transformers import BertTokenizer,BertModel, BertConfig
from sklearn.metrics import classification_report,f1_score,accuracy_score,recall_score,precision_score
import torch

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/7.25

ValueError: mount failed

In [ ]:
import torch
import datetime
from transformers import BertModel, BertTokenizer, BertConfig

# 获取当前日期字符串
current_date = datetime.datetime.now().date().strftime("%Y%m%d")

class Config(object):
    def  __init__(self):
        self.model_name = 'bert'
        self.train_path = 'train_bert.txt'
        self.dev_path = 'val_bert.txt'
        self.test_path = 'test_bert.txt'

        #类别文档
        self.type_path = 'class.txt'
        #类别名称
        self.type_name = [line.strip() for line in open(self.type_path,encoding='utf-8')]

        # 模型训练+预测的时候  训练设备，如果GPU可用，则为cuda，否则为cpu
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.bert_path = "/content/drive/MyDrive/bert/bert-base-chinese"  # 预训练BERT模型的路径
        self.bert_model = BertModel.from_pretrained(self.bert_path)  # 加载预训练BERT模型
        self.tokenizer = BertTokenizer.from_pretrained(self.bert_path)  # BERT模型的分词器
        self.bert_config = BertConfig.from_pretrained(self.bert_path)  # BERT模型的配置

        #模型训练参数
        self.num_classes = len(self.type_name)
        self.max_len = 32
        self.batch_size = 128
        self.num_epochs = 2
        self.learning_rate = 1e-5




In [ ]:
from tqdm import tqdm
import torch
from torch.utils.data import Dataset,DataLoader
from transformers import BertTokenizer, TextDataset


def load_data(path):
    """
    加载数据 数据已经处理好了
    :return:
    """
    result = []
    with open(path,encoding='utf-8') as f:
        for line in tqdm(f,desc='loading data'):
            line = line.strip()
            if not line:
                continue
            label,text = line.split('\t',1)

            label = int(label)
            result.append((text,label))
        return result

class textDataset(Dataset):
    def __init__(self,data_list):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):
        text,label = self.data_list[index]
        return text,label

def data_loader():
    #读取原始数据
    train_data = load_data(config.train_path)
    valid_data = load_data(config.dev_path)
    test_data = load_data(config.test_path)

    #构建数据集
    train_dataset = textDataset(train_data)
    valid_dataset = textDataset(valid_data)
    test_dataset = textDataset(test_data)

    #构建数据加载器
    train_data = DataLoader(dataset=train_dataset,batch_size=config.batch_size,shuffle=True,collate_fn=collate_fun)
    valid_data = DataLoader(dataset=valid_dataset,batch_size=config.batch_size,shuffle=True,collate_fn=collate_fun)
    test_data = DataLoader(dataset=test_dataset,batch_size=config.batch_size,shuffle=True,collate_fn=collate_fun)

    return train_data,valid_data,test_data

def collate_fun(batch):
    texts,labels = zip(*batch)
    text_tokens = config.tokenizer.batch_encode_plus(
        texts,
        truncation=True,
        max_length=config.max_len,
        padding="max_length",
        return_tensors="pt",
        return_attention_mask=True
    )
    input_ids = text_tokens["input_ids"]
    attention_mask = text_tokens["attention_mask"]
    labels = torch.tensor(labels)
    return input_ids,attention_mask,labels


In [ ]:
import torch
from tqdm import tqdm
from sklearn.metrics import classification_report,f1_score,accuracy_score,precision_score,recall_score

def vailModel(model,vail_loader,device):
    model.eval()
    all_preds,all_labels = [],[]
    with torch.no_grad():
        for i,batch in enumerate(tqdm(vail_loader,desc="Vailing")):
            input_ids,attention_mask,labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            #前向传播
            output = model(input_ids,attention_mask)
            y_preds = torch.argmax(output,dim=1)

            all_preds.extend(y_preds.tolist())
            all_labels.extend(labels.tolist())
        report = classification_report(all_labels,all_preds)
        f1 = f1_score(all_labels,all_preds,average='macro')
        acc = accuracy_score(all_labels,all_preds)
        pre = precision_score(all_labels,all_preds,average='macro')
        recall = recall_score(all_labels,all_preds,average='macro')
        return report,f1,acc,pre,recall


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

config = Config()
#定义bert模型
class MyBertClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained("/content/drive/MyDrive/bert/bert-base-chinese")
        #定义全连接层fc,用于分类任务
        self.fc = nn.Linear(config.bert_config.hidden_size,config.num_classes)

    def forward(self,input_ids,attention_mask):
        output = self.bert(input_ids,attention_mask)
        logits = self.fc(output.pooler_output)
        return logits

In [ ]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import f1_score,accuracy_score
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

def trainModel():
    #准备数据
    train_data,valid_data,test_data = data_loader()
    #准备模型
    model = MyBertClassifier()
    model.to(config.device)
    #优化器和损失函数
    optimizer = AdamW(model.parameters(),lr=config.learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    #开始训练模型
    best_f1 = 0.0
    for epoch in range(config.num_epochs):
        model.train()
        total_loss= 0.0
        train_preds,train_labels = [],[]
        #定义datafrem存储训练结果
        loss_df = pd.DataFrame()
        for i,batch in enumerate(tqdm(train_data,desc='training....')):
            #将数据移动到设备上
            input_ids,attention_mask,labels = batch
            input_ids = input_ids.to(config.device)
            attention_mask = attention_mask.to(config.device)
            labels = labels.to(config.device)

            #前向传播
            output = model(input_ids,attention_mask)
            #计算损失
            loss = loss_fn(output,labels)
            total_loss += loss.item()
            y_pred_list = torch.argmax(output,dim=1)
            #存储日志
            train_preds.extend(y_pred_list.tolist())
            train_labels.extend(labels.tolist())

            #梯度清零
            optimizer.zero_grad()
            #反向传播
            loss.backward()
            #参数更新
            optimizer.step()

            #每10批计算训练指标
            if (i+1) % 10 == 0:
                train_f1 = f1_score(train_labels,train_preds,average='macro')
                acc = accuracy_score(train_labels,train_preds)
                batch_count = i %10 +1
                avg_loss = total_loss / batch_count
                #将avg_loss添加到dataframe中
                loss_df['batch_loss'] = [avg_loss]
                #打印训练日志
                print(f"epoch:{epoch+1},batch:{i+1},loss:{avg_loss:.4f},train_f1:{train_f1:.4f},acc:{acc:.4f}")
                #清空累计损失和真实标签
                total_loss = 0.0
                train_preds,train_labels = [],[]

            #每100个批次，验证模型
            if (i+1) % 100 == 0:
                report, f1score, accuracy, precision, recall = vailModel(model, valid_data, config.device)
                print("验证集评估报告：\n", report)
                print(
                    f"验证集的f1: {f1score:.4f}, accuracy:{accuracy:.4f}, precision:{precision:.4f}, recall:{recall:.4f}")
                # todo 将模型再设置为训练模式
                model.train()
                # todo 如果验证F1分数优于历史最佳，保存模型
                if f1score > best_f1:
                    # 更新历史最佳F1分数
                    best_f1 = f1score
                    # 保存模型
                    torch.save(model.state_dict(),"./model/mymodel.pt")
                    print("保存模型成功, 当前f1分数:", best_f1)

        loss_df.to_csv("./model/los_log.csv", index=False)



In [ ]:
trainModel()

loading data: 280682it [00:02, 110471.94it/s]
loading data: 15325it [00:00, 23606.93it/s]
loading data: 15325it [00:00, 29378.87it/s]
training....:   0%|          | 10/2193 [00:06<21:16,  1.71it/s]

epoch:1,batch:10,loss:2.2524,train_f1:0.0817,acc:0.1133


training....:   1%|          | 20/2193 [00:12<20:52,  1.74it/s]

epoch:1,batch:20,loss:2.1194,train_f1:0.2056,acc:0.2422


training....:   1%|▏         | 30/2193 [00:17<21:04,  1.71it/s]

...

training....:  99%|█████████▉| 2180/2193 [34:00<00:08,  1.61it/s]

epoch:2,batch:2180,loss:0.4758,train_f1:0.8524,acc:0.8516


training....: 100%|█████████▉| 2190/2193 [34:06<00:01,  1.62it/s]

epoch:2,batch:2190,loss:0.4772,train_f1:0.8564,acc:0.8570


training....: 100%|██████████| 2193/2193 [34:08<00:00,  1.07it/s]
